In [279]:
import csv
import json
import re
import unicodedata
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Set, Tuple

In [176]:
import numpy as np

In [177]:
import pandas as pd
pd.set_option('display.max_rows', 1000)  # 省略されないように表示件数を設定

In [178]:
def define_main_sets(df: pd.DataFrame) -> Tuple[Set[str], Set[int], Set[str], Set[str], Set[str]]:
    """
    各集合を定義する関数.
    """
    # 授業のリスト
    subjects = df['授業コード'].unique().tolist()
    
    # 時限のリスト
    weekdays = ['月', '火', '水', '木', '金', '土']
    times = [str(t) for t in range(1, 6)]
    periods = [w + t for w in weekdays for t in times]
    
    # 教室のリスト
    facility_path = 'materials/misc/facility.csv'
    df_facility = pd.read_csv(facility_path)
    rooms = df_facility['施設名'].str.cat(df_facility['講義室'], sep='_').unique().tolist()
    
    # クラスの集合
    # 厳密ではないが、まずはコースをクラスと考える
    cs = df['ｺｰｽ'].unique().tolist()
    courses = set(sum([pattern.findall(c) for c in cs], []))
    
    # 担当教員の集合
    teachers = set(sum(df['teachers'].tolist(), []))
    teachers.discard('師玉')  # おそらく師玉真or師玉礼だと思われるので削除
    
    return set(subjects), set(periods), set(rooms), courses, teachers

In [301]:
def define_main_dicts(df: pd.DataFrame) -> Tuple[Dict[str, List[str]], Dict[str, List[str]], Dict[str, List[str]]]:
    """
    各辞書を定義する関数.
    
    # 授業の辞書{code: [授業名,科目担当,コマ数,クラス]}
    # 科目担当の辞書{teacher: lecture_list}
    # クラス授業の辞書{class: lecture_list}
    """
    subject_propaties = defaultdict(list)
    teacher_lectures = defaultdict(list)
    course_lectures = defaultdict(list)
    for code in subjects:
        data = df[df['授業コード'] == code]
        _, lecture_name, _, _, duration, _courses, _, _, _, _teachers = data.values[0]
        # 講義名の英数字は半角にカタカナは全角化、不要な空白は除去
        lecture_name = unicodedata.normalize('NFKC', lecture_name.title().strip())
        # 授業辞書
        subject_propaties[code] = [lecture_name, _teachers, duration, _courses]
        # 科目担当辞書
        for teacher in _teachers:
            teacher_lectures[teacher].append(lecture_name)
        # クラス授業辞書
        cs = set(sum([pattern.findall(c) for c in _courses], []))
        for c in cs:
            course_lectures[c].append(lecture_name)
    
    return subject_propaties, teacher_lectures, course_lectures

In [6]:
DATA_DIR = Path('../data/')
SAVE_DIR = DATA_DIR.joinpath('csv')
files_path = list(DATA_DIR.joinpath('timetable_2023').glob('*.xlsx'))

In [302]:
dfs = {}
pattern = re.compile('[A-Z]')
for file_path in files_path:
    df = pd.read_excel(file_path, sheet_name=1)                                   # 0番目(X科)は時間割の神エクセル、1番目(Sheet1)がシラバス形式
    df['授業コード'] = df['授業コード'].astype(str).apply(lambda x: x.zfill(4))   # 授業コードは左ゼロ埋めで4桁
    df['開講期'] = df['開講期'].astype(np.int8)
    df['必選'] = df['必選'].astype(str).apply(lambda x: x.replace('○◎', '◎'))  # 「○◎」はよくわからんのでいったん必須(◎)とする
    df['コマ数'] = df['コマ数'].astype(np.int8)
    df['ｺｰｽ'] = df['ｺｰｽ'].fillna('all').astype(str).apply(lambda x: x.strip())    # \u3000(全角スペース)が含まれているので除去
    df['学年'] = df['学年'].astype(str).apply(lambda x: x.replace('・', '&'))     # 数値と文字列が混在しているので文字列に統一
    df['ｸﾗｽ'] = df['ｸﾗｽ'].fillna('all')                                           # 何も記載がないものは全員が対象(all)とする
    key = pattern.findall(str(file_path))[0]
    dfs[key] = df
df = dfs['M']
cols = df.columns

# 整形
classカラム:「学年_クラス_コース」を一つの組と考える  
roomカラム: 「建物名_部屋番号」を一つの部屋と考える  
teachersカラム: 講師陣のリスト

In [303]:
df.head()

,授業コード,授業科目,開講期,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
0,0001,スタディスキル,1,◎,1,M・F・E・T,1,④-1,高橋明,NaN
1,0002,スタディスキル,1,◎,1,M・F・E・T,1,④-2,福山,NaN
2,0003,スタディスキル,1,◎,1,M・F・E・T,1,④-3,福山,NaN
3,0004,スタディスキル,1,◎,1,M・F・E・T,1,④-4,小林,NaN
4,0029,（特）スタディスキル,1,◎,1,M・F・E・T,2,all,小田切,NaN


In [304]:
# # TODO: 0790, 1075, 1096, 8006, 8053, 8057のクラスを決めるのが厄介
# id_duplicates = df[df.duplicated(subset=('授業コード'))]['授業コード']
# df[df['授業コード'].isin(id_duplicates)]

In [305]:
# 授業コードが重複しているサンプルに対して、コースを統合
id_duplicates = df[df.duplicated(subset=('授業コード'))]['授業コード']
for id_dup in id_duplicates:
    targets = (df['授業コード'] == id_dup)
    cs = df[df['授業コード'] == id_dup]['ｺｰｽ'].tolist()
    # 重複している講義のコースリストからコース(大文字アルファベット一文字)を抜き出して重複コースを削除
    course_set = set(sum([pattern.findall(c) for c in cs], []))
    # 先頭のみを指定しようとしてilocでアクセスすると警告なしのChain Indexingで元のdfが変更されないので両方とも変更
    df.loc[targets, ['ｺｰｽ']] = '・'.join(list(course_set))

# 授業コードが重複しているレコードを削除
# 1096: 機械工学プロジェクトⅡと1097: 機械工学プロジェクトⅡは同じカラムがあり、後ろの方にのみ講師情報があるので後ろを残す
df.drop_duplicates(subset=('授業コード'), inplace=True, keep='last')
df.shape

(429, 10)

In [306]:
# 教員が存在しない講義(例: Academic English for Global leaders系)はわからないのでいったん除去
df = df.dropna(subset=['教員名']).copy()
df.shape

(425, 10)

In [307]:
# 正規表現で無駄な記号や空白を除去
garbage = re.compile('[!"#$%&\'\\\\()*+-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]')
teachers_raw = df['教員名'].tolist()
teachers_clean = [re.sub(garbage, ',', s).replace('\u3000', '').strip(',').strip() for s in teachers_raw]

df['teachers'] = [tc.split(',') for tc in teachers_clean]
df = df.drop(columns=['教員名'])

# 解空間を狭める
## 1.開講期ごとに分割

In [308]:
df_first = df[df['開講期'] == 1]
df_second = df[df['開講期'] == 2]
df_through = df[df['開講期'] == 3]

assert len(df_first) + len(df_second) + len(df_through) == len(df)

print(len(df_first))
print(len(df_second))
print(len(df_through))

227
196
2


## 2-A.学年ごとに分割

In [309]:
df_first_1 =  df_first[df_first['学年'].str.contains('1')]
df_first_2 =  df_first[df_first['学年'].str.startswith('2')]
df_first_3 =  df_first[df_first['学年'].str.startswith('3')]
df_first_4 =  df_first[df_first['学年'] == '4']

assert len(df_first_1) + len(df_first_2) + len(df_first_3) + len(df_first_4) == len(df_first)

print(len(df_first_1))
print(len(df_first_2))
print(len(df_first_3))
print(len(df_first_4))

56
107
61
3


## 2-B.必須と選択に分割

In [310]:
df_required = df_first[df_first['必選'] == '◎']
df_required_select = df_first[df_first['必選'] == '□']
df_option = df_first[df_first['必選'] == '○']
df_req_sel_re = df_first[df_first['必選'] == '■']

assert len(df_required) + len(df_required_select) + len(df_option) + len(df_req_sel_re) == len(df_first)

print(len(df_required))
print(len(df_required_select))
print(len(df_option))
print(len(df_req_sel_re))

79
20
127
1


# 集合の定義

In [311]:
subjects, periods, rooms, courses, teachers = define_main_sets(df_required)

print(f'subjects: {len(subjects)}, {subjects}')
print(f'periods: {len(periods)}, {periods}')
print(f'subjects: {len(rooms)}, {rooms}')
print(f'courses: {len(courses)}, {courses}')
print(f'teachers: {len(teachers)}, {teachers}')

subjects: 79, {'9014', '1057', '0002', '1004', '1048', '9052', '0087', '1002', '0003', '1007', '0568', '9045', '9015', '1089', '0088', '8279', '9019', '0070', '1042', '0071', '1073', '0103', '0505', '8053', '9007', '0105', '1044', '8094', '1078', '0566', '1026', '1046', '1024', '9013', '0520', '0474', '0472', '8095', '9017', '0001', '1106', '8139', '1009', '9018', '1013', '1084', '9024', '9051', '1006', '1031', '0090', '9023', '9035', '1047', '0101', '0230', '8275', '1003', '1074', '1008', '9025', '0232', '1030', '9046', '0820', '9001', '1109', '0792', '0029', '0470', '1014', '0490', '1079', '1102', '0004', '9042', '1051', '1023', '0476'}
periods: 30, {'水4', '火3', '土2', '木1', '月2', '金1', '火1', '土5', '火5', '月3', '木4', '水3', '土1', '水5', '火4', '水1', '金4', '金5', '土3', '水2', '金3', '月1', '木2', '金2', '木3', '木5', '土4', '火2', '月5', '月4'}
subjects: 72, {'K3号館_3102', 'K1号館_1201', 'K1号館_604', 'K2号館_1403', 'K3号館_3301', 'K3号館_3001', 'K1号館_303', 'K3号館_3502', 'K3号館_3308', 'B5号館_2106', 'K2号館_1408', 'K1

# 辞書の定義

In [312]:
subject_propaties, teacher_lectures, course_lectures = define_main_dicts(df_required)

print(subject_propaties['0001'])
print(teacher_lectures['高橋明'])  # TODO: 講義は別なので倫理学の名前の扱いをどうするか
print(course_lectures['M'])

['スタディスキル', ['高橋明'], 1, 'M・F・E・T']
['スタディスキル']
['制御工学', 'スタディスキル', '流体力学', '材料力学II', 'スタディスキル', '流れ学I', '現代社会講座', '材料工学', '現代社会講座', '機械設計法I', '日本国憲法', '英会話I', '線形代数学I-B', '日本国憲法', '材料工学', '機械系数学', '機械製図基礎', '機械工学概論', '機械力学I', '英会話II', 'スタディスキル', '熱力学I', '創造設計ユニットI', '生産加工学', 'プログラミング基礎', '機械工学概論', '日本国憲法', '健康・スポーツ科学実習I', '材料力学II', '機械設計法I', '流れ学I', '健康・スポーツ科学実習I', 'プログラミング基礎', 'キャリア設計', '情報リテラシー', '(特)スタディスキル', '熱力学I', '科学技術英語I', '機械製図基礎', 'スタディスキル', '機械応用実験', '機械力学I']


# 制約の定義

In [313]:
# 教室のキャパ行列
facility_path = 'materials/misc/facility.csv'
df_facility = pd.read_csv(facility_path)
df_facility['施設名'] = df_facility['施設名'].str.cat(df_facility['講義室'], sep='_').unique().tolist()
room_capacity = dict(zip(df_facility['施設名'], df_facility['最大席数']))
room_capacity['K1号館_B101']

108

## sr_map: 授業sが教室rで利用可能かを表す01行列

In [314]:
# 講義数の重複数で定員数を割った値で人数の欠損値を補完
## M科: 機械工学コースの入学時の定員は180人(ref. https://op.kait.jp/admission/recruit/)
num_max = 180
dup_count = df_required.pivot_table(index=['授業科目'], aggfunc='size').to_dict()
df_required.loc[:, '最大人数'] = df_required.apply(lambda df: num_max / dup_count[df['授業科目']]
                                        if pd.isnull(df['最大人数']) else df['最大人数'], axis=1)
df_required.head()

,授業コード,授業科目,開講期,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,最大人数,teachers
0,0001,スタディスキル,1,◎,1,M・F・E・T,1,④-1,45.0,[高橋明]
1,0002,スタディスキル,1,◎,1,M・F・E・T,1,④-2,45.0,[福山]
2,0003,スタディスキル,1,◎,1,M・F・E・T,1,④-3,45.0,[福山]
3,0004,スタディスキル,1,◎,1,M・F・E・T,1,④-4,45.0,[小林]
4,0029,（特）スタディスキル,1,◎,1,M・F・E・T,2,all,180.0,[小田切]


In [315]:
num_subjects = len(df_required)
num_rooms = len(df_facility)
sr_map = defaultdict(lambda: np.zeros(num_rooms, dtype=np.int8))

# 行が授業、列が部屋の01行列を作成
room_cap = df_facility['最大席数'].values.astype(int)
sr_mat = np.tile(room_cap, (num_subjects, 1))
sr_mat = (df_required['最大人数'].values <= sr_mat.T).T.astype(np.int8)  # ブロードキャストするため、増やす次元を先頭にしてから再度戻す
assert sr_mat.shape == (num_subjects, num_rooms)

# 授業コードをkey, その部屋群が使用可(1) or 不可(0)をvalueとする辞書
for i, subject in enumerate(df_required['授業コード']):
    sr_map[subject] = sr_mat[i]

## pr_map: 時限pに教室rが利用可能かを表す01行列

In [316]:
pr_map = defaultdict(lambda: np.ones(num_rooms, dtype=np.int8))
for p in periods:
    pr_map[p] = np.ones(num_rooms, dtype=np.int8)

## tp_map: 教員tが時限pに授業可能かを表す01行列

In [317]:
num_periods = len(periods)
tp_map = defaultdict(lambda: np.ones(num_periods, dtype=np.int8))
for t in teachers:
    tp_map[t] = np.ones(num_periods, dtype=np.int8)

## sp_map: 授業sが時限pに実施可能かを表す01行列

In [318]:
sp_map = defaultdict(lambda: np.ones(num_periods, dtype=np.int8))
for s in subjects:
    sp_map[s] = np.ones(num_periods, dtype=np.int8)

## cp_map: コースcが時限pに授業可能かを表す01行列

In [319]:
cp_map = defaultdict(lambda: np.ones(num_periods, dtype=np.int8))
for c in courses:
    cp_map[c] = np.ones(num_periods, dtype=np.int8)

### 各種ファイルを保存

In [320]:
# 集合をcsvで保存
save_dir = SAVE_DIR.joinpath('first', 'constrains')
save_dir.mkdir(parents=True, exist_ok=True)

file_names = ['subjects', 'periods', 'rooms', 'courses', 'teachers']
sets = [subjects, periods, rooms, courses, teachers]
for file_name, data in zip(file_names, sets):
    save_path = save_dir.parent.joinpath(f'{file_name}.csv')
    with open(save_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(sorted(list(data)))

In [321]:
# 授業情報をjsonで保存
dict_names = ['subject_propaties', 'teacher_lectures', 'course_lectures']
dicts = [subject_propaties, teacher_lectures, course_lectures]
for file_name, data in zip(dict_names, dicts):
    save_path = save_dir.parent.joinpath(f'{file_name}.json')
    with open(save_path, 'wt', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [333]:
# 制約をjsonで保存
file_names = ['sr_map', 'pr_map', 'tp_map', 'sp_map', 'cp_map']
constrains = [sr_map, pr_map, tp_map, sp_map, cp_map]
for file_name, data in zip(file_names, constrains):
    # NumPy配列はjson保存に非対応のためリストに変換
    for key, value in data.items():
        data[key] = value.tolist()
    save_path = save_dir.joinpath(f'{file_name}.json')
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)